# U-Net 논문 정리

## Patch Learning
<img src = "https://t1.daumcdn.net/cfile/tistory/99304833605C18F313?original">  

- U-Net은 중복되지 않은 patch를 이용해 학습  
- 이는 patch가 겹칠 때 불필요한 연산이 늘어나는 것을 방지하기 위함  
- 또한 속도도 느리고 시간도 오래 걸림  

## Overlap-tile  
<img src = "https://t1.daumcdn.net/cfile/tistory/99269846605C18F311?original">  

- 위에서 말했듯이 patchwise 학습을 이용한다    
- 높은 해상도의 이미지를 출력하기 위해 overlap-tile을 이용한다    
- U-Net의 down-sampling 과정에서 padding을 사용하지 않기 때문에 output의 사이즈가 input보다 작음  
- 이 논문에서는 mirroring padding이라는 것을 이용함  
- patch 단위로 잘라 네트워크에 입력하는데 노란색 영역을 기준으로 사이드에 이미지를 symmetric해서 채움  

## U-Net architecture
<img src = "https://t1.daumcdn.net/cfile/tistory/997C3A42605C18F311?original">  

- U-Net architecture는 크게 Contracting Path와 Expansive Path로 나뉘어짐  
- 1. Contractign Path  
    - 전형적인 convolution network 구조  
    - 두 번의 3x3 convolution을 반복 수행하며 padding을 사용하지 않음  
    - 활성화 함수로는 ReLU를 사용  
    - 2x2 max-pooling과 stride 2를 사용  
    - down-sampling 과정에서 feature channel이 2배씩 늘어남  
- 2. Expansive Path  
    - 2x2 up-convolution을 사용  
    - up-sampling 과정에서 feature channel이 2배씩 줄어듦  
    - contracting path에서 max-pooling 되기 전의 feature map의 테두리 부분을 사이즈에 맞게 crop하여 up-convolution할 때 concatenation을 진행  
    - 두 번의 3x3 convolution을 반복  
    - 활성화 함수로는 ReLU를 사용  
- 3. Final Layer  
    - 마지막 layer에서는 1x1 convolution을 사용해 2개의 클래스로 분류  
    - object와 background 두 개의 클래스를 구분하기 위해 2개의 클래스로 지정함

## Optimizer And Loss  
- optimizer로는 stochastic gradient descent  
- 이때 momentum의 값을 0.99를 줘서 이전의 값들을 많이 반영하게 함  

**Softmax function**
<img src = "https://miro.medium.com/max/594/0*qLhMYSprvsobuot1.png">  
**Cross Entropy Loss with $w(x)$ function**  
<img src = "https://miro.medium.com/max/700/0*8kUKwV1z9DWX9B0k.png">  
<img src = "https://miro.medium.com/max/700/0*s2oMdVI8w1ms9N0l.png">  
- 이를 논문에서는 Energy function이라고 정의함  
- $d_1$과 $d_2$는 $x$에서 가장 가까운 세포까지의 거리, 두 번째로 가까운 세포까지의 거리를 계산하는 함수  
- 즉, $x$는 세포 사이에 존재하는 픽셀이며 두 세포 사이의 간격이 좁을수록 weight를 큰 값으로 부여한다  
- 이를 다음 그림으로 나타냄  

<img src = "https://t1.daumcdn.net/cfile/tistory/997DE543605C18F311?original">

## Data Augmentation  
- U-Net은 적은 데이터로 충분히 학습하기 위해 data augmentation을 사용함  
- 일반적인 목적이 아니라 biomedical에서 쓰이기 위한 목적으로 만들어졌으므로 data augmentation이 나쁘지 않은 성능을 가져올 수 있다고 판단  
- 또한 elastic deformation을 통해 세포가 변형되는 것까지 augmentation을 진행

## Code Implementation Pipeline  
- 1. data augmentation을 진행 (elastic deformation을 포함)  
- 2. U-Net architecture와 똑같은 네트워크 생성  
- 3. Momentum 및 Energy function 정의  
- 4. 사용자 정의 모델 학습으로 훈련 진행